Symplectic数値解法によるTime-depend Schrödinger方程式
==========================================================

NumPy・SciPyを利用してSymplectic解法を実装します. 

In [1]:
# coding: utf-8

import os
import re
import numpy as np
from scipy.integrate import quad
from scipy.fftpack import fft
from scipy.fftpack import ifft
from inspect import getsource

`Import`
--------

### `os`
shellを呼んだりなんだりと, OS環境の操作が可能. 後でGnuplotを呼んだりするのに用いる. 

### `re`
正規表現ライブラリ. 文字列操作に便利.

### `numpy`
高速なArrayを提供. 演算に関する高速に動作するメソッドも多く持っているので, それを活用しつついかにfor文を書かないかが勝負になる. 

### `scipy.integrate.quad`
積分. 積分範囲を`[-np.inf, np.inf]`とかにできるなど, 仕様がとっても便利な上にかなり高速. Cで適当なコードを書くより全然早かった. 恐らく内部では関数のタイプによってアルゴリズムを変えるような仕組みがある模様(適応刻み幅とか). 

### `scipy.fftpack.fft / ifft`
フーリエ変換と逆フーリエ変換. これまた速度はかなり優秀なはず. 

### `inspect`
関数の中身の情報を取得できるライブラリ. `getsource`は関数のソースコードを`string`で返すメソッド. 後々Gnuplotに渡すデータを細工するのに用いる. 




In [2]:
# --*-- Set functions --*--

# Set potential
# Note1 : Active line must be attached semicolon(;) at the end of line
# Note2 : Use the math function which work on gnuplot as well without any change
def Potential(x):
    #return x**2
    return 0.01*(x**2 - 8*abs(x) + 16);

# Potential on exp
def V(x):
    return -0.5j*Potential(x)

# Initial function
def Psi_0(x):
    y = lambda z: np.exp(-2*(z-7)**2)
    return y(x)/quad(y, -np.inf, np.inf)[0]/2

# file writer
def file_writer(filename, arr_func):
    with open(filename, "w") as f:
        for n, x in enumerate(np.linspace(-L/2, L/2, N)):
            print("{0}\t{1}".format(x, abs(arr_func[n])), file=f)

関数定義
----------------------------------------
### `Potential(x):`
系のポテンシャル. 後でGnuplotにもこの関数を渡さなければいけないが, ベタ打ちすると`Potential`を変えたときにGnuplot側も書き換えなければいけないので, `inspect`ライブラリを用いてこれを簡略化する. そのとき上のようにコメントアウトした部分とアクティブな部分を区別するため, アクティブな行には文末にセミコロン(;)を付けるようにする. また, 用いる関数はGnuplotでもそのまま使えるもののみを使うように. 例えば
```python
import numpy as np
def Potential(x):
    np.sin(x)
```
ではGnuplotで呼べないので, 
```python
from numpy import sin
def Potential(x):
    sin(x)
```
のようにすること. 
### `V(x):`
Symplecticのexpの上に乗っかるやつ. 
### `Psi_0(x):`
初期関数. ガウシアンにしている. 規格化っぽいことをしてます. 
### `file_writer(filename, arr_func):`
ファイル書き込みを関数化. ファイル名と書き込むデータを引数に取る. あんまり知られてないけど, `print`関数は
```python
print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
```

という仕様. デフォルトで`python file=sys.stdout`を持っているので, `file=f`とかにすればファイル書き込みに`print`が使える(一般にはよく`write`とか`writeline`関数が用いられるみたい). `end=""`とかにすれば改行を飛ばせるし, `flush`もできるし, 結構優秀. 


In [3]:
# --*-- Set constants and variables for initial condition --*--

# Space step, Volume, Time step
N, L, dt = 512, 25.0, 0.5

# Set expK
expK = np.array([])
for n in range(0, N):
    if(n < N/2):
        expK = np.append(expK, np.exp(-0.5j*(2*np.pi*n/N)**2*dt))
    else:
        expK = np.append(expK, np.exp(-0.5j*(2*np.pi*(n-N)/N)**2*dt))

# Set expV, initial function arr_Psi_old
expV ,arr_Psi = np.array([]), np.array([], dtype=complex)
for x in np.linspace(-L/2, L/2, N):
    expV = np.append(expV, np.exp(V(x)*dt))
    arr_Psi = np.append(arr_Psi, Psi_0(x))

定数定義
--------------------
### `N`, `L`, `dt`
`N`は空間分割数, `L`は体積(系の長さ), `dt`は時間刻み幅
### expK
Symplectic解法における運動項の時間発展演算子
$$\exp\left(\frac{1}{2}ik^2\Delta t\right)$$
x-空間をFourier変換したので, kは
$$ k = \frac{2\pi n}{N}\ \ (n<N/2)\ \ or\ \ \frac{2\pi (N-n)}{N}\ \ (n> N/2) $$
であることに注意. 
### arr_Psi, expV
arr_Psiは時間発展させたい関数を格納. 最初はPsi_0を入れることになる. 後々`complex type`のexpVを掛け合わせることになるので, そのことを考えて`type=complex`を付けています. 

expVはx-空間で掛けるものなので普通に格納. 



In [4]:
#--*-- Time propagation by symplectic numerical solution --*--

# Time length, Output interbal, Total file piece
times, total_file = 40000, 500
M = times//total_file
for i in range(times):
    # output time depend function
    if(i%M == 0):
        file_writer("output{0}.txt".format(i//M), arr_Psi)
    
    # Multipling by time propagator of potential term
    arr_Psi *= expV
    
    # Fourier transformation
    arr_Psi = fft(arr_Psi)
    
    # Multipling time propagator of kinetic term
    arr_Psi *= expK
    
    # Inverse fourier transformation
    arr_Psi = ifft(arr_Psi)


Symplectic時間発展
------------------------
コレがこのコードのメインディッシュなんですが, やってることはホントにそのまんま. こんなかんじにスマートに書けるのが`Python`のいいところだと思います. 

`numpy.array()`だと, 掛け算は要素ごとの掛け算になり, `return`も`numpy.array()`になります. Python標準の`List`だったら
```python
for n, i in enumerate(expV):
    arr_Psi[n] *= i
```
みたいな`for`文が必要になるところ. `numpy`だから高速に動作するのでいいね. 上のコードだと2重ループになるのでアウトです. 


In [5]:
# --*-- Gnuplot --*--

# Get potential function as string
pattern = re.compile("(return.+;)")
m = pattern.search(getsource(Potential))
str_potential = str(m.group(0))[7:-1]

# System call for gnuplot
gnuplot_call = 'gnuplot -e '
start = '"'
set_range = 'set xr[{0}:{1}]; set yr[{2}:{3}]; '.format(-L/2, L/2, 0, 0.5)
plot_initial = 'plot \\"output0.txt\\" w l lw 2 title \\"t = 0\\",{0} title \\"potential\\" w l lw 2; pause 2; '.format(str_potential)
do_for_declaration = 'do for[i = {0}:{1}:1]'.format(1, total_file-1)
do_for_start = "{"
do_for_procedure = 'plot sprintf(\\"output%d.txt\\", i) title sprintf(\\"t = %d\\", i) w l lw 2, {0} title \\"potential\\" w l lw 2; pause 0.05;'.format(str_potential)
end = '}"'

os.system("".join([gnuplot_call, start, set_range, plot_initial, do_for_declaration, do_for_start, do_for_procedure, end]))

os.system("rm output*.txt")

0

出力 - Gnuplot
------------------------

最後はGnuplotで動画をつくります. `matplotlib`でもいいんですが, なかなか動画を作るのが面倒なので. `os.system()`でshellが呼べるのでそれでGnuplotを直接呼んでます. `range`の変数とか`do for`のレンジとかは`format`文で指定してます. この辺はPython埋め込みが便利なところで, もしpltファイルを別に用意したら, `L`なり`total_file`なりが変わったときに,pltファイルの方も修正しなければならないのがとても面倒です. 

同様の理由からポテンシャルの関数も変数として用意してあげたい. というわけで`inspect.getsource()`を使って`Potential(x)`のソースコードを取得しています. そこから正規表現を用いて`return`から`;`までの部分を切り取ってあげてます. 
```python
str_potential = str(m.group(0))[7:-1]
```
は`return`と`;`を切り取るようなスライスです. これで`str_potential`にはポテンシャル関数が`string`で格納されました. 

出力ファイルを沢山生成しているので最後に
```python
os.system("rm output*.txt")
```
で全削除. 1ファイルだけにまとめることもできますが, そうするとファイルサイズが大きくなりすぎてGnuplotの読み込みが遅くなります. 動画を作るときはファイルは分割したほうが良いみたいです. 